In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import skimage
from scipy import misc
from skimage import io
from skimage.morphology import disk, watershed
from skimage.filters import threshold_otsu, rank, gaussian, threshold_local, threshold_sauvola
from skimage.util import img_as_ubyte
from skimage.transform import probabilistic_hough_line, hough_circle, hough_circle_peaks
from skimage.draw import line, circle, polygon, circle_perimeter
from skimage import feature
from skimage.filters import gaussian
from skimage.morphology import binary_erosion, binary_opening, binary_closing, binary_dilation, convex_hull_object
from skimage.segmentation import clear_border, random_walker
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
import copy

from sklearn.ensemble import IsolationForest

%matplotlib inline

## 1. Equalizing histograms

In [234]:
def equalize_hist(filename):
    raw_img = sp.misc.imread("./Train/" + filename[:-5] + ".bmp")

    img_enh_contr = skimage.exposure.equalize_adapthist(raw_img / 256.0)
    

    plt.imsave("./Train/Equalized_hist/" + filename, img_enh_contr)
    
    return img_enh_contr

## 2. Further actions

### 1. Loading

In [4]:
# filename = "Pict_4_1.tiff"
# raw_img = sp.misc.imread("./Train/Equalized_hist/" + filename, mode="RGB")

# img = img_as_ubyte(raw_img)

# radius = 15
# selem = disk(radius)
# local_otsu = rank.otsu(img[:, :, 0], selem)
# threshold_global_otsu = threshold_otsu(img[:, :, 0])
# img_otsu = np.ones_like(img[:, :, 0]) * (img[:, :, 0] >= threshold_global_otsu) * 255

# plt.imshow(img[:,:,0], cmap="gray")
# plt.show()
# plt.imshow(img[:,:,1], cmap="gray")
# plt.show()
# plt.imshow(img[:,:,2], cmap="gray")
# plt.show()
# print(img_otsu)
# plt.imshow(img_otsu, cmap="gray")
# plt.show()

### 2. Canny detector

In [5]:
def Canny(img):
    canny = np.zeros_like(img)
    canny[:, :, 0] = feature.canny(img[:, :, 0], low_threshold=120, high_threshold=150) # 100 130
    canny[:, :, 1] = feature.canny(img[:, :, 1], low_threshold=120, high_threshold=150) # 80 150
    canny[:, :, 2] = feature.canny(img[:, :, 2], low_threshold=120, high_threshold=150) # 80 110

    plt.imsave("./Train/Canny/" + filename[:-5] + "_R.tiff", canny[:, :, 0], cmap="gray")
    plt.imsave("./Train/Canny/" + filename[:-5] + "_G.tiff", canny[:, :, 1], cmap="gray")
    plt.imsave("./Train/Canny/" + filename[:-5] + "_B.tiff", canny[:, :, 2], cmap="gray")

    return canny

### 3. Hough Transform

In [221]:
def Hough(canny):
    lines = list(range(3))
    lines[0] = probabilistic_hough_line(canny[:, :, 0], threshold=62, line_length=60, line_gap=11) # threshold 62 length 40 gap 10
    lines[1] = probabilistic_hough_line(canny[:, :, 1], threshold=52, line_length=60, line_gap=11) # threshold 52
    lines[2] = probabilistic_hough_line(canny[:, :, 2], threshold=72, line_length=60, line_gap=11) # threshold 72

    # Sort
    lines_new = copy.deepcopy(lines)
    lines_new[0] = sorted(lines[0], key=lambda tup: tup[0][0])
    lines_new[1] = sorted(lines[0], key=lambda tup: tup[0][0])
    lines_new[2] = sorted(lines[0], key=lambda tup: tup[0][0])

    lines = lines_new

    for dim in range(3):
        for num, l in enumerate(lines[dim]):
            if l[1][0] < l[0][0]:
                lines[dim][num] = (l[1], l[0])



    blank_img_R = np.zeros_like(img)
    blank_img_G = np.zeros_like(img)
    blank_img_B = np.zeros_like(img)

    for l in lines[0]:
        rr, cc = line(l[0][1], l[0][0], l[1][1], l[1][0])
        blank_img_R[rr, cc, 0] = 255
    for l in lines[1]:
        rr, cc = line(l[0][1], l[0][0], l[1][1], l[1][0])
        blank_img_G[rr, cc, 1] = 255
    for l in lines[2]:
        rr, cc = line(l[0][1], l[0][0], l[1][1], l[1][0])
        blank_img_B[rr, cc, 2] = 255

    plt.imsave("./Train/Lines/" + filename[:-5] + "_R.tiff", blank_img_R)
    plt.imsave("./Train/Lines/" + filename[:-5] + "_G.tiff", blank_img_G)
    plt.imsave("./Train/Lines/" + filename[:-5] + "_B.tiff", blank_img_B)
    
    return lines

### 4. Find triangles in lists of lines

In [7]:
length_threshold_low = 70
length_threshold_high = 130
closeness_threshold = 20 # 20
closeness_threshold_centers = 50
angle_threshold = 0.1


def check_length(lengths, dim, line_num):
    if lengths[dim][line_num] > length_threshold_low and lengths[dim][line_num] < length_threshold_high:
        return True
    else:
        return False
    
    
def check_closeness(lines, dim, line_num_1, line_num_2):
    flag = False
    closeness = -1
    i_best = -1
    j_best = -1
    
    for i in range(2):
        for j in range(2):
            temp = ((lines[dim][line_num_1][i][0] - lines[dim][line_num_2][j][0]) ** 2 +
                    (lines[dim][line_num_1][i][1] - lines[dim][line_num_2][j][1]) ** 2) ** 0.5
            
            if temp < closeness_threshold:
                if flag == False:
                    closeness = copy.deepcopy(temp)
                    i_best = copy.deepcopy(i)
                    j_best = copy.deepcopy(j)

                    flag = True
                else:
                    pass
#                     print("Lines are parallel !?")
    return (closeness, i_best, j_best)


def check_angle(lines, lengths, dim, line_num_1, line_num_2):#, i_best, j_best):
    vec_1 = (lines[dim][line_num_1][1][0] - lines[dim][line_num_1][0][0],
             lines[dim][line_num_1][1][1] - lines[dim][line_num_1][0][1])
    vec_2 = (lines[dim][line_num_2][1][0] - lines[dim][line_num_2][0][0],
             lines[dim][line_num_2][1][1] - lines[dim][line_num_2][0][1])
    
    scalar_product = vec_1[0] * vec_2[0] + vec_1[1] * vec_2[1]
    
    cos = np.fabs(scalar_product / (lengths[dim][line_num_1] * lengths[dim][line_num_2]))

    if (np.fabs(cos - 0.5) < angle_threshold): # 0.5 = cos(pi / 3)
#         print(np.arccos(cos) / 3.14 * 180)
        return True
    else:
        return False


def delete_identical_triangles(triangles):
    triangles_new = []
    for triangle in triangles:
        angle_1 = triangle[0][triangle[2][0]]
        angle_2 = triangle[1][triangle[2][1]]
        angle = (np.int(np.floor((angle_1[0] + angle_2[0]) * 0.5)), np.int(np.floor((angle_1[1] + angle_2[1]) * 0.5)))
        
#         print(angle_1, angle_2, angle)
        flag = True
        for triangle_to_comp in triangles_new:
            if ((triangle_to_comp[1][0] - angle[0]) ** 2 +
                (triangle_to_comp[1][1] - angle[1]) ** 2) ** 0.5 < closeness_threshold:
                flag = False
                break
                
        if flag == True:
            triangles_new.append((triangle[0][1 - triangle[2][0]], angle, triangle[1][1 - triangle[2][1]]))
    return triangles_new
    
    
def delete_obtuse_angles(triangles):
    triangles_new = []
    for triangle in triangles:
        vec_1 = (triangle[0][0] - triangle[1][0], triangle[0][1] - triangle[1][1])
        vec_2 = (triangle[2][0] - triangle[1][0], triangle[2][1] - triangle[1][1])
        
        length_1 = (vec_1[0] ** 2 + vec_1[1] ** 2) ** 0.5
        length_2 = (vec_2[0] ** 2 + vec_2[1] ** 2) ** 0.5
        
        scalar_product = vec_1[0] * vec_2[0] + vec_1[1] * vec_2[1]
    
        cos = scalar_product / (length_1 * length_2)
#         print(np.arccos(cos) / 3.14 * 180)
        
        if cos > 0:
            triangles_new.append(triangle)
        else:
            pass
#             print("Obtuse angle!")
            
    return triangles_new
    
    
def find_centers(triangles):
    centers = []
    
    for triangle in triangles:
        center = (np.int(np.floor((triangle[0][0] + triangle[1][0] + triangle[2][0]) / 3)),
                  np.int(np.floor((triangle[0][1] + triangle[1][1] + triangle[2][1]) / 3)))
        centers.append(center)
        
    return centers
    
    
def check_close_centers(triangles, centers):
    centers_new = []
    triangles_new = []
    
    for num, center in enumerate(centers):
        flag = True
        for num2, center_to_comp in enumerate(centers_new):
            if ((center_to_comp[0] - center[0]) ** 2 +
                (center_to_comp[1] - center[1]) ** 2) ** 0.5 < closeness_threshold_centers:
                flag = False
                
                # Update center coordinates
                centers_new[num2] = (np.int(np.floor((center[0] + center_to_comp[0]) * 0.5)),
                                    np.int(np.floor((center[1] + center_to_comp[1]) * 0.5)))
                
                break
                
        if flag == True:
            centers_new.append((center[0], center[1]))
            triangles_new.append(triangles[num])
    return triangles_new, centers_new
    
    
def find_triangles(lines):
    lengths = copy.deepcopy(lines)
    triangles = []
    
    for dim in range(3):
        for num, l in enumerate(lines[dim]):
            lengths[dim][num] = ((l[0][1] - l[1][1]) ** 2 + (l[0][0] - l[1][0]) ** 2) ** 0.5
    
    for dim in range(3):
        for line_num_1 in range(len(lines[dim])):
            for line_num_2 in range(line_num_1 + 1, len(lines[dim])):
                # Check length of lines, their closeness and angle between them
                if check_length(lengths, dim, line_num_1) * check_length(lengths, dim, line_num_2) *\
                    (check_closeness(lines, dim, line_num_1, line_num_2)[0] >= 0) *\
                    check_angle(lines, lengths, dim, line_num_1, line_num_2) == True:
#                     print("Yes! Triangle!")
                    triangles.append((lines[dim][line_num_1], lines[dim][line_num_2],
                                     check_closeness(lines, dim, line_num_1, line_num_2)[1:]))
                     
                
    triangles = delete_identical_triangles(triangles)
    
    triangles = delete_obtuse_angles(triangles)
    
    centers = find_centers(triangles)
    
    triangles, centers = check_close_centers(triangles, centers)
    
    return triangles, centers

In [8]:
min_dist_threshold = 50


def dist_between_points(point_1, point_2):
    return ((point_1[0] - point_2[0]) ** 2 + (point_1[1] - point_2[1]) ** 2) ** 0.5


def dist_between_triangles(triangle_1, triangle_2):
    dist = np.zeros(9)
    for i in range(3):
        for j in range(3):
            dist[3 * i + j] = dist_between_points(triangle_1[i], triangle_2[j])
            
    # Find minimum of sums of distances between 3 points
    for i in range(9):
        for j in range(i + 1, 9):
            for k in range(j + 1, 9):
                
                if i == 0 and j == 1 and k == 2:
                    min_dist = dist[i] + dist[j] + dist[k]
                else:
                    cur_dist = dist[i] + dist[j] + dist[k]
                    if cur_dist < min_dist:
                        min_dist = cur_dist
                        
    return min_dist


def check_same_triangles(triangles):
    triangles_new = []
    counts = []
    
    triangles_new.append(triangles[0])
    counts.append(1)
    
    for num, triangle in enumerate(triangles[1:]):
        flag_counted = False
        for num2, triangle_to_comp in enumerate(triangles_new):
            if dist_between_triangles(triangle, triangle_to_comp) < min_dist_threshold:
                counts[num2] = counts[num2] + 1
                
                flag_counted = True
                break # Now check next input triangle
        
        if flag_counted == False:
            triangles_new.append(triangle)
            counts.append(1)
            
    print(len(triangles_new))
    print(counts)
    
    return triangles_new, counts

In [9]:
num_of_points = 100
radius = 20
activation_std_threshold = 0.17
std_coef = 0.9


def check_colour_features(centers, img, triangles):
    std = []
    triangles_new = copy.deepcopy(triangles)
    centers_new = copy.deepcopy(centers)
    
    for center in centers:
        xx = np.floor(np.random.uniform(low=center[0] - radius, high=center[0] + radius, size=num_of_points)).astype(int)
        yy = np.floor(np.random.uniform(low=center[1] - radius, high=center[1] + radius, size=num_of_points)).astype(int)

        # Reds, Greens, Blues
#         points = skimage.color.rgb2gray(img)[yy, xx]
        points = img[yy, xx, 0]
        std.append(np.std(points))
        
    std = np.array(std) / 255.0
    print(np.array(std))
    
    max_std = np.amax(std)
    
    if max_std < activation_std_threshold:
        pass
    else:
#         std_threshold = std_coef * max_std #########  !!!!!!!!!!!!!!!!!!!!!!!!!
        std_threshold = activation_std_threshold
        
        bad_numbers = list(np.nonzero(std > std_threshold)[0])
        if len(bad_numbers) > 0:
            for num in bad_numbers[-1::-1]:
                del triangles_new[num]
                del centers_new[num]
                std[num] = -1
                
    print(np.array(std))
        
    return triangles_new, centers_new

In [10]:
coef_enlarge = 0 # !


def build_triangles_on_centers_angles(triangles, centers):
    triangles_new = []
    centers_new = []
    for num in range(len(centers)):
        median_dir = (centers[num][0] - triangles[num][1][0], centers[num][1] - triangles[num][1][1])
        
        median_end = (centers[num][0] + 0.5 * median_dir[0],
                      centers[num][1] + 0.5 * median_dir[1])

        
        # Normalize
#         print(median_dir)
        third_side_dir = ((-median_dir[1] / median_dir[0]) / ((1 + (median_dir[1] / median_dir[0]) ** 2) ** 0.5),
                          1.0 / ((1 + (median_dir[1] / median_dir[0]) ** 2) ** 0.5))
        
#         print(third_side_dir[0] * median_dir[0] + third_side_dir[1] * median_dir[1])
        
        # Enlarge the triangle
        length_median = ((median_end[0] - (triangles[num][1][0] - median_dir[0] * coef_enlarge)) ** 2 +\
                         (median_end[1] - (triangles[num][1][1] - median_dir[1] * coef_enlarge)) ** 2) ** 0.5
        
        length_side = 2 / np.sqrt(3.0) * length_median
#         length_side = 1.5 * length_median
        
        second_angle = (np.int(np.floor(median_end[0] - third_side_dir[0] * length_side / 2)),
                        np.int(np.floor(median_end[1] - third_side_dir[1] * length_side / 2)))
        third_angle = (np.int(np.floor(median_end[0] + third_side_dir[0] * length_side / 2)),
                       np.int(np.floor(median_end[1] + third_side_dir[1] * length_side / 2)))
        
        # Enlarge
        triangles_new.append((second_angle, (np.int(np.floor((triangles[num][1][0] - median_dir[0] * coef_enlarge))),
                                             np.int(np.floor((triangles[num][1][1] - median_dir[1] * coef_enlarge)))), third_angle))
        centers_new.append((np.int(np.floor((second_angle[0] + triangles_new[num][1][0] + third_angle[0]) / 3.0)),
                          np.int(np.floor((second_angle[1] + triangles_new[num][1][1] + third_angle[1]) / 3.0))))

#         print(triangles[num])
#         print(triangles_new[num])
        
    return triangles_new, centers_new

In [250]:
def cut_triangles(triangles, centers):
    polygons = []
    for num in range(len(triangles)):
        median_dir_1 = (centers[num][0] - triangles[num][0][0], centers[num][1] - triangles[num][0][1])
        median_end_1 = (triangles[num][0][0] + median_dir_1[0] * 1.5, triangles[num][0][1] + median_dir_1[1] * 1.5)
        
        median_dir_2 = (centers[num][0] - triangles[num][1][0], centers[num][1] - triangles[num][1][1])
        median_end_2 = (triangles[num][1][0] + median_dir_2[0] * 1.5, triangles[num][1][1] + median_dir_2[1] * 1.5)
        
        median_dir_3 = (centers[num][0] - triangles[num][2][0], centers[num][1] - triangles[num][2][1])
        median_end_3 = (triangles[num][2][0] + median_dir_3[0] * 1.5, triangles[num][2][1] + median_dir_3[1] * 1.5)
        
        polygon_1 = (triangles[num][0], median_end_2, centers[num], median_end_3)
        polygon_2 = (triangles[num][1], median_end_1, centers[num], median_end_3)
        polygon_3 = (triangles[num][2], median_end_1, centers[num], median_end_2)
        
        polygons.append((polygon_1, polygon_2, polygon_3))
        
    return polygons


def find_nearest_colour(mean_h, mean_s, mean_v):
    colours = {"white": 1, "black": 2, "yellow": 3, "blue": 4, "red": 5}
    
    white_s = 0 # 1
    white_v = 255 # 1
    
#     green_h = 110 # 2
    green_h = 85 # 2
    green2_h = 125 # 2
    black_v = 0 # 2
    
    yellow_h = 55 # 3
    orange_h = 30 # 3
    
    blue_h = 230 # 4
    
    red_h = 0 # 5
    red2_h = 360 # 5
    
    # Black
#     print("Mean_v: ", mean_v)
    if mean_v < 30: # 25
#         print("Green")
        return colours["black"]
    
    # White
#     print("Mean_v: ", mean_v, "Mean_s: ", mean_s)
    if mean_v > 160 and mean_s < 0.4:
#         print("White")
        return colours["white"]
    
    # Other
    dist_to_green = np.minimum(np.fabs(mean_h - green_h), np.fabs(mean_h - green2_h))
    dist_to_yellow = np.minimum(np.fabs(mean_h - yellow_h), np.fabs(mean_h - orange_h))
    dist_to_blue = np.fabs(mean_h - blue_h)
    dist_to_red = np.minimum(np.fabs(mean_h - red_h), np.fabs(mean_h - red2_h))
    
    t = np.array([dist_to_green, dist_to_yellow, dist_to_blue, dist_to_red])
#     print(t)
    
    colour = np.argmin(t)
    
    if colour == 0:
#         print("Green")
        return colours["black"] # green == black
    elif colour == 1:
#         print("Yellow")
        return colours["yellow"]
    elif colour == 2:
#         print("Blue")
        return colours["blue"]
    elif colour == 3:
#         print("Red")
        return colours["red"]
    

def count_balls(polygons, img):
    balls_count = []
    balls_coords = []
    
    for p in polygons:
        balls_count_cur = []
        balls_coords_cur = []
        for i in range(3):
            rr, cc = polygon(np.array((p[i][0][1], p[i][1][1], p[i][2][1], p[i][3][1])),
                             np.array((p[i][0][0], p[i][1][0], p[i][2][0], p[i][3][0])))

#             print(rr.shape)
#             print(cc.shape)
            
            min_row = np.amin(rr)
            max_row = np.amax(rr)
            min_col = np.amin(cc)
            max_col = np.amax(cc)
            
#             print(min_row, max_row, min_col, max_col)
            
            polygon_img = np.zeros((max_row - min_row + 1, max_col - min_col + 1, 3))       
            polygon_img[rr - min_row, cc - min_col, :] = img[rr, cc, :]
            
            mask = polygon_img[:, :, 0] > 0 # !!!!!!!!!!!!!
            mask = binary_erosion(mask, np.ones((5, 5)))
            
            polygon_img_copy = copy.deepcopy(polygon_img)
            polygon_img_copy[:, :, 0][mask == 0] = 0
            polygon_img_copy[:, :, 1][mask == 0] = 0
            polygon_img_copy[:, :, 2][mask == 0] = 0
            
            # HSV!
            polygon_img = skimage.color.rgb2hsv(polygon_img)
            polygon_img_copy = skimage.color.rgb2hsv(skimage.filters.gaussian(polygon_img_copy, sigma=0.2, multichannel=True))
#             print(polygon_img)
            
#             plt.imshow(skimage.color.hsv2rgb(polygon_img) / 255.0)
#             plt.show()
#             plt.imshow(skimage.color.hsv2rgb(polygon_img_copy) / 255.0)
#             plt.show()
            
            
            
            binarized_img = copy.deepcopy(polygon_img[:, :, 1])
            binarized_img[mask == 0] = 0
            
#             plt.imshow(binarized_img, cmap="Greys")
#             plt.show()
        
    
    
    
    
    
            X = np.zeros((binarized_img.shape[0], binarized_img.shape[1], 3))
            X = copy.deepcopy(polygon_img_copy)
            X = X.reshape((-1, 3))
            
            isolation_forest = IsolationForest(n_estimators=500, contamination=0.2, bootstrap=True)
            isolation_forest.fit(X)
            decision_func_img = isolation_forest.decision_function(X).reshape((binarized_img.shape[0], binarized_img.shape[1]))
#             plt.imshow(decision_func_img, cmap="gray")
#             plt.show()
    
            decision_func_img = (decision_func_img - np.min(decision_func_img)) / (np.max(decision_func_img) - np.min(decision_func_img))
    
            threshold = threshold_otsu(decision_func_img[np.where(mask)])
            binarized_dec_func = decision_func_img > threshold
            
#             plt.imshow(binarized_dec_func, cmap="gray")
#             plt.show()
    
#             closed_binarized_dec_func = binary_closing(binarized_dec_func, selem=np.ones((3, 3)))
            closed_binarized_dec_func = skimage.filters.median(binarized_dec_func, selem=np.ones((3, 3)), mask=mask)
            closed_binarized_dec_func = binary_closing(closed_binarized_dec_func, selem=np.ones((2, 2)))
#             plt.imshow(closed_binarized_dec_func, cmap="gray")
#             plt.show()
        
            # Isolation Forest!!!
            opened_binarized_img = closed_binarized_dec_func
            
#             plt.imshow(opened_binarized_img, cmap="gray")
#             plt.show()
            
    
            label_img = label(np.logical_not(opened_binarized_img))
            image_label_overlay = label2rgb(label_img)
#             plt.imshow(image_label_overlay)
#             plt.show()
            
            tmp_count = 0
            for region in regionprops(label_img):     
#             for region in regionprops(clusters_img):
#                 print(region.area, region.equivalent_diameter, region.eccentricity)


                # Isolation Forest!!!
                if region.area > 11 and region.area < 75 and region.equivalent_diameter > 4.8 and\
                region.equivalent_diameter < 9.5 and region.eccentricity < 0.76:
                    tmp_count += 1
#                     print(region.centroid, region.equivalent_diameter / 2)
                    balls_coords_cur.append((np.int(np.round(region.centroid[1])),
                                             np.int(np.round(region.centroid[0])), np.int(np.floor(region.equivalent_diameter / 2) + 1)))
#                     print(balls_coords_cur)
                    

                    center = region.centroid
                    radius = region.equivalent_diameter * 0.5
                    rr, cc = circle(np.int(np.round(center[1])), np.int(np.round(center[0])), np.int(np.floor(radius)))
                    
                    
#                     img_copy2 = copy.deepcopy(polygon_img)
#                     img_copy2 = skimage.color.hsv2rgb(img_copy2) / 255.0
#                     for num in range(len(balls_coords_cur)):
#                         rr, cc = circle(np.int(np.round(center[1])), np.int(np.round(center[0])), np.int(np.floor(radius)))
#                         img_copy2[cc, rr, 1] = 1
#                     plt.imshow(img_copy2)
#                     plt.show()    

                    mean_h = np.median(polygon_img[:, :, 0][cc, rr]) * 360
                    mean_s = np.median(polygon_img[:, :, 1][cc, rr])
                    mean_v = np.median(polygon_img[:, :, 2][cc, rr])

#                     print(mean_h, mean_s, mean_v)
                    
                    tmp_count = find_nearest_colour(mean_h, mean_s, mean_v)
                    
                    break
#             print(tmp_count)
            if tmp_count == 0:
                balls_coords_cur.append((0, 0, 0))
            
            balls_count_cur.append(tmp_count)
         
    
#             img_copy2 = copy.deepcopy(polygon_img)
#             img_copy2 = skimage.color.hsv2rgb(img_copy2) / 255.0
            
#             for num in range(len(balls_coords_cur)):
#                 rr, cc = circle(balls_coords_cur[num][0], balls_coords_cur[num][1], balls_coords_cur[num][2])
#                 img_copy2[cc, rr, 1] = 1
            
#             plt.imshow(img_copy2)
#             plt.show()

        balls_coords.append((balls_coords_cur)) 

        balls_count.append(balls_count_cur)
    
    return (balls_count, balls_coords)

### 5. Save results

In [245]:
def save_results(img, triangles, centers, polygons, balls_coords, balls_count, suffix):
    blank_img2 = np.zeros_like(img)
    img_copy = copy.deepcopy(img)

    for l in triangles:
        rr, cc = line(l[0][1], l[0][0], l[1][1], l[1][0])
        blank_img2[rr, cc, 0] = 255
        img_copy[rr, cc, 0] = 255
        # First point
        rr, cc = circle(l[0][1], l[0][0], 3)
        blank_img2[rr, cc, 0] = 255
        img_copy[rr, cc, 0] = 255

        rr, cc = line(l[1][1], l[1][0], l[2][1], l[2][0])
        blank_img2[rr, cc, 1] = 255
        img_copy[rr, cc, 1] = 255
        # Second point
        rr, cc = circle(l[1][1], l[1][0], 3)
        blank_img2[rr, cc, 1] = 255
        img_copy[rr, cc, 1] = 255

        # Third point
        rr, cc = circle(l[2][1], l[2][0], 3)
        blank_img2[rr, cc, 2] = 255
        img_copy[rr, cc, 2] = 255

    for c in centers:
        rr, cc = circle(c[1], c[0], 5)
        blank_img2[rr, cc, 2] = 255
        img_copy[rr, cc, 2] = 255

    for num, p in enumerate(polygons):
        for i in range(3):
            rr, cc = polygon(np.array((p[i][0][1], p[i][1][1], p[i][2][1], p[i][3][1])),
                             np.array((p[i][0][0], p[i][1][0], p[i][2][0], p[i][3][0])))
#             blank_img2[rr, cc, i] = 100
#             img_copy[rr, cc, i] = 100
            
            min_row = np.amin(rr)
            min_col = np.amin(cc)
            
#             print(balls_coords)
#             print(balls_count)
#             print(num, i)

#             rr, cc = circle_perimeter(b[1], b[0], b[2] + 2)
#             img_copy[min_row + rr, min_col + cc, :] = 255

            b = balls_coords[num][i]
            rr, cc = circle(b[1], b[0], 5)
#             print(min_row, min_col)
#             print(rr, cc)
#             print(min_row + rr, min_col + cc)
#             print(balls_count[num][i])

            if balls_count[num][i] == 1: # white
                img_copy[min_row + rr, min_col + cc, :] = 255
            elif balls_count[num][i] == 2: # green
                img_copy[min_row + rr, min_col + cc, 1] = 255
            elif balls_count[num][i] == 3: # yellow
                img_copy[min_row + rr, min_col + cc, 0] = 255
                img_copy[min_row + rr, min_col + cc, 1] = 255
            elif balls_count[num][i] == 4: # blue
                img_copy[min_row + rr, min_col + cc, 2] = 255
            elif balls_count[num][i] == 5: # red
                img_copy[min_row + rr, min_col + cc, 0] = 255
                
    # plt.imshow(blank_img2)
    # plt.imshow(img_copy)
    # plt.show()

    plt.imsave("./Train/Triangles/prefinal_" + filename[:-5] + "_" + suffix + filename[-5:], img_copy)
    
    return

### 6. Saving results to file

In [243]:
def save_to_file(filename, centers, balls_count):
    out = open("./Train/Triangles/" + filename[:-5] + ".txt", "w")
    
#     print(len(centers))
    out.write(str(len(centers)) + "\n")
    for i in range(len(centers)):
#         print("%d, %d; %d %d %d" % (centers[i][0], centers[i][1], balls_count[i][0], balls_count[i][1], balls_count[i][2]))
        out.write("%d, %d; %d %d %d\n" % (centers[i][0], centers[i][1], balls_count[i][0], balls_count[i][1], balls_count[i][2]))
    out.close()

### 7. Testing

In [251]:
num_samples = 20 # 20 !!!


filenames = ["Pict_1_1.tiff", "Pict_1_2.tiff", "Pict_2_1.tiff", "Pict_2_2.tiff", "Pict_2_3.tiff",
             "Pict_3_1.tiff", "Pict_3_2.tiff", "Pict_4_1.tiff", "Pict_4_2.tiff"]
# filenames = ["for_report_2.tiff"]#, "for_report_1.tiff"]
# filenames = filenames[5:]
# filenames = filenames[6:7]

correct_number_of_triangles = [11, 8, 13, 9, 9, 10, 8, 10, 14]
# correct_number_of_triangles = [3]#, 3]
# correct_number_of_triangles = correct_number_of_triangles[5:]
# correct_number_of_triangles = correct_number_of_triangles[6:7]

for num, filename in enumerate(filenames):
# for num, filename in enumerate(filenames_temp):
#     raw_img = sp.misc.imread("./Train/Equalized_hist/" + filename, mode="RGB")
    raw_img = equalize_hist(filename)

    img = img_as_ubyte(raw_img)

    canny = Canny(img)

    #####
    all_triangles = []
    all_centers = []
    for i in range(num_samples):
        
        lines = Hough(canny)

        triangles, centers = find_triangles(lines)
#         print(triangles)
#         print()
        all_triangles = all_triangles + triangles
        all_centers = all_centers + centers
        
    triangles = all_triangles
    centers = all_centers
    
    
    triangles, counts = check_same_triangles(triangles)
    
    centers = find_centers(triangles)
    
    triangles, centers = check_close_centers(triangles, centers)
    
    triangles, centers = check_colour_features(centers, img, triangles)
    
#     triangles, centers = build_triangles_on_centers_angles(triangles, centers)
    
    
    polygons = cut_triangles(triangles, centers)
    
        
    # Here for test. Needs to be deleted
#     save_results(img, triangles, centers, polygons, "before")
#     triangles, centers = build_triangles_on_centers_angles(triangles, centers)

    
    balls_count, balls_coords = count_balls(polygons, img)
#     print("Here: ", balls_coords)
#     print("There: ", balls_count)
    
    if len(triangles) == correct_number_of_triangles[num]:
        cor = "Correct"
    else:
        cor = "Error!"
    
    print(filename + " --- Triangles: " + str(len(triangles)) + " / " +\
          str(correct_number_of_triangles[num]) + " --- " + cor)
    print("Numbers of balls:")
    print(balls_count)
    print()
    
    save_results(img, triangles, centers, polygons, balls_coords, balls_count, "after")
    save_to_file(filename, centers, balls_count)

C:\Program Files (x86)\Anaconda\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
C:\Program Files (x86)\Anaconda\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


3
[20, 20, 20]
[ 0.08972469  0.08542664  0.09228853]
[ 0.08972469  0.08542664  0.09228853]
for_report_2.tiff --- Triangles: 3 / 3 --- Correct
Numbers of balls:
[[5, 5, 5], [3, 5, 0], [2, 3, 4]]

